In [76]:
import sys
import os

# Get the absolute path to the project directory
project_dir = os.path.abspath("..")

# Append the project directory to sys.path
if project_dir not in sys.path:
    sys.path.append(project_dir)

In [77]:
import sys
import os

# Get the absolute path to the project directory
project_dir = os.path.abspath("..")

# Append the project directory to sys.path
if project_dir not in sys.path:
    sys.path.append(project_dir)

import yfinance as yf
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import msgpack
import warnings
from dataclasses import asdict, is_dataclass, dataclass
import os
import msgpack
import json
import pickle
from pandas.api.types import is_datetime64_any_dtype
import scipy
import numpy as np
from scipy.stats import linregress

from src.common.AssetData import AssetData
from src.common.AssetDataService import AssetDataService
from src.common.AssetFileInOut import AssetFileInOut 
from src.databaseService.OutsourceLoader import OutsourceLoader

from src.simulation.SimulatePortfolio import SimulatePortfolio
from src.strategy.StratBuyAndHold import StratBuyAndHold
from src.simulation.ResultAnalyzer import ResultAnalyzer
from src.common.AssetFileInOut import AssetFileInOut
from src.mathTools.CurveAnalysis import CurveAnalysis
from src.predictionModule.FourierML import FourierML
from src.common.YamlTickerInOut import YamlTickerInOut
from src.common.DataFrameTimeOperations import DataFrameTimeOperationsPandas as DFTO


In [78]:
from keras.datasets import mnist

# Load data
(XTrain, YTrain), (XTest, YTest) = mnist.load_data()

# Use the first 5000 samples
XTrain = XTrain[:5000]
YTrain = YTrain[:5000]
XTest = XTest[:5000]
YTest = YTest[:5000]

print(XTrain.shape)
print(YTrain.shape)
print(XTest.shape)

# One-hot encode labels
Y = np.zeros((5000, 10))
Ytest = np.zeros((5000, 10))
labels = np.arange(10)
for n in range(5000):
    Y[n, YTrain[n]] = 1
    Ytest[n, YTest[n]] = 1

# Reshape images to (n_samples, 28*28)
U = XTrain.reshape(5000, 28*28)
Util = XTest.reshape(5000, 28*28)

# Parameters
NRnd = 2000

MeanAccDATA = np.zeros(1)
gammaDATA = np.zeros(1)
varDATA = np.zeros(1)

(5000, 28, 28)
(5000,)
(5000, 28, 28)


In [ ]:
# Generate random complex matrix W
W_real = np.random.randn(NRnd, 28*28)
W_imag = np.random.randn(NRnd, 28*28)
W = W_real + 1j * W_imag
# Unbias W
UnbiasedW = W - np.mean(W)

# Normalize real and imaginary parts
NormalizedRealW = UnbiasedW.real / np.sqrt(np.var(UnbiasedW.real))
NormalizedImagW = UnbiasedW.imag / np.sqrt(np.var(UnbiasedW.imag))
VarW = np.var(UnbiasedW.real) + 1j * np.var(UnbiasedW.imag)
print(f'StdDevW = {np.sqrt(VarW.real)} + {np.sqrt(VarW.imag)}j')
Wnor = (NormalizedRealW + 1j * NormalizedImagW) / np.sqrt(NRnd)

StdDevW = 0.9991388949222844 + 0.9998824724630789j


In [90]:
# Machine Learning
XMat = np.abs(np.matmul(Wnor, U.T)).T  # Shape: (5000, NRnd)
XtilMat = np.abs(np.matmul(Wnor, Util.T)).T  # Shape: (5000, NRnd)

gammaData = np.linspace(10, 11, 100)
qualityData = np.zeros(len(gammaData))

In [ ]:
XtX = np.matmul(U.T, U)

eigs = np.linalg.eigvals(XtX)

sorteigs= np.sort(eigs)
sorteigs

In [93]:
ga = -100
XtX = np.matmul(XMat.T, XMat) + ga * np.eye(NRnd)
XtY = np.matmul(XMat.T, Y)
coeffs = np.linalg.solve(XtX, XtY)
Ytil = np.matmul(XtilMat, coeffs)

estIdxLabel = np.argmax(Ytil, axis=1)
estLabel = estIdxLabel
accuracy_per_label = np.zeros(len(labels))
for q in range(len(labels)):
    indices = np.where(YTest == labels[q])[0]
    accuracy_per_label[q] = np.mean(estLabel[indices] == labels[q])

np.mean(accuracy_per_label)

C:\Users\KILightTouch\AppData\Local\Temp\ipykernel_15540\3952747847.py:2: RuntimeWarning: invalid value encountered in matmul
  XtX = np.matmul(XMat.T, XMat) + ga * np.eye(NRnd)


0.9033574877941483

In [84]:
for g in range(len(gammaData)):
    print(f'g = {g+1} / {len(gammaData)}')
    gamma = gammaData[g]
    XtX = np.matmul(XMat.T, XMat) + gamma * np.eye(NRnd)
    XtY = np.matmul(XMat.T, Y)
    coeffs = np.linalg.solve(XtX, XtY)
    Ytil = np.matmul(XtilMat, coeffs)
    
    estIdxLabel = np.argmax(Ytil, axis=1)
    estLabel = estIdxLabel
    accuracy_per_label = np.zeros(len(labels))
    for q in range(len(labels)):
        indices = np.where(YTest == labels[q])[0]
        accuracy_per_label[q] = np.mean(estLabel[indices] == labels[q])
    qualityData[g] = np.mean(accuracy_per_label)
    print(qualityData[g])

g = 1 / 100


C:\Users\KILightTouch\AppData\Local\Temp\ipykernel_15540\1931645022.py:4: RuntimeWarning: invalid value encountered in matmul
  XtX = np.matmul(XMat.T, XMat) + gamma * np.eye(NRnd)


0.901835123618611
g = 2 / 100
0.901835123618611
g = 3 / 100
0.901835123618611
g = 4 / 100
0.901835123618611
g = 5 / 100
0.901835123618611
g = 6 / 100
0.901835123618611
g = 7 / 100
0.901835123618611
g = 8 / 100
0.901835123618611
g = 9 / 100


KeyboardInterrupt: 

In [66]:
bestIdx = np.argmax(qualityData)
bestGamma = gammaData[bestIdx]
print(f'Best gamma: {bestGamma}')

# Recompute with best gamma
gamma = bestGamma
XtX = np.dot(XMat.T, XMat) + gamma * np.eye(NRnd)
XtY = np.dot(XMat.T, Y)
coeffs = np.linalg.solve(XtX, XtY)
Ytil = np.dot(XtilMat, coeffs)
estIdxLabel = np.argmax(Ytil, axis=1)
estLabel = estIdxLabel

accuracy_per_label = np.zeros(len(labels))
for q in range(len(labels)):
    indices = np.where(YTest == labels[q])[0]
    accuracy_per_label[q] = np.mean(estLabel[indices] == labels[q])
    print(f'Accuracy label {labels[q]}: {accuracy_per_label[q]}')
print(f'Accuracy: {np.mean(accuracy_per_label)}')
print(bestGamma)
print(np.mean(accuracy_per_label))

Best gamma: 10.0
Accuracy label 0: 0.9565217391304348
Accuracy label 1: 0.978984238178634
Accuracy label 2: 0.8981132075471698
Accuracy label 3: 0.898
Accuracy label 4: 0.906
Accuracy label 5: 0.8662280701754386
Accuracy label 6: 0.9155844155844156
Accuracy label 7: 0.88671875
Accuracy label 8: 0.8548057259713702
Accuracy label 9: 0.8692307692307693
Accuracy: 0.9030186915818232
10.0
0.9030186915818232
